<a href="https://colab.research.google.com/github/nosadchiy/public/blob/main/InventoryMDPNestedFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
from scipy.optimize import minimize

###############################################################################
# 1. Define the Inventory MDP Class
###############################################################################

class InventoryBackorderMDP:
    """
    A single-product inventory model with backorders.

    States (i): integer on-hand inventory (i >= 0) or negative for backorders.
                We truncate to range [i_min, i_max].
    Actions (Q): discrete order quantity (0..Q_max).

    Demand: Poisson(lambda_) by default, but can be replaced with any pmf.
    Costs:
       - K * indicator(Q>0)   (fixed order cost)
       - c * Q                (per-unit ordering cost)
       - h * max(i', 0)       (holding cost on leftover inventory)
       - b * max(-i', 0)      (backorder cost if i' < 0)
    i' = i + Q - D is the next inventory level after demand.

    This MDP is solved by value iteration with a discount factor beta.
    """
    def __init__(self,
                 i_min=-5,       # smallest state we track (backorder limit)
                 i_max=10,       # largest state we track
                 Q_max=5,        # max order quantity
                 lambda_=2.0,    # demand mean if using Poisson
                 beta=0.95,      # discount factor
                 K=5.0,          # fixed order cost
                 c=1.0,          # per-unit ordering cost
                 h=0.5,          # holding cost per unit leftover
                 b=2.0           # backorder cost per unit short
                ):
        self.i_min = i_min
        self.i_max = i_max
        self.Q_max = Q_max
        self.lambda_ = lambda_
        self.beta = beta
        # cost parameters
        self.K = K
        self.c = c
        self.h = h
        self.b = b

        # Build the integer state space
        self.state_space = np.arange(i_min, i_max + 1)  # e.g. [-5, -4, ..., 10]
        self.n_states = len(self.state_space)
        # Build the discrete action space
        self.action_space = np.arange(Q_max + 1)  # e.g. [0, 1, 2, 3, 4, 5]

        # Precompute Poisson pmf up to some upper bound
        # We pick something larger than i_max + Q_max - i_min to be safe
        self.demand_max = max(0, i_max - i_min + Q_max + 5)
        pmf_vals = [self.poisson_pmf(k, self.lambda_) for k in range(self.demand_max+1)]
        self.demand_pmf = np.array(pmf_vals)

        # Precompute transitions & expected immediate cost if we want to speed up
        # (For large state spaces, this might be big. Here, let's do it for demonstration.)
        self.trans_probs = {}
        self.cost_cache = {}
        for s_idx, i_val in enumerate(self.state_space):
            for Q in self.action_space:
                # Build distribution over next states
                # i' = i_val + Q - d
                # Probability that next state = j for each j in state_space
                prob_vec = np.zeros(self.n_states)
                exp_cost = 0.0
                for d in range(self.demand_max + 1):
                    prob_d = self.demand_pmf[d]
                    i_next = i_val + Q - d
                    # clamp i_next into [i_min, i_max]
                    if i_next < self.i_min:
                        i_next = self.i_min
                    elif i_next > self.i_max:
                        i_next = self.i_max
                    next_idx = i_next - self.i_min  # index shift

                    # immediate cost from realized demand d
                    cost_d = self.one_period_cost(i_val, Q, d)
                    exp_cost += prob_d * cost_d

                    prob_vec[next_idx] += prob_d

                self.trans_probs[(s_idx, Q)] = prob_vec
                self.cost_cache[(s_idx, Q)] = exp_cost

    def one_period_cost(self, i_val, Q, d):
        """Returns the cost in a single period, given state i_val, order Q, and realized demand d."""
        # After ordering Q, inventory becomes i_val + Q.
        # Then demand d is subtracted: i' = i_val + Q - d.
        # The realized cost:
        #  1) fixed cost if Q>0
        #  2) per-unit ordering cost c * Q
        #  3) holding cost h * max(i', 0)
        #  4) backorder cost b * max(-i', 0)
        K_cost = self.K if Q > 0 else 0.0
        c_cost = self.c * Q
        i_prime = i_val + Q - d
        hold_cost = self.h * max(i_prime, 0)
        back_cost = self.b * max(-i_prime, 0)
        return K_cost + c_cost + hold_cost + back_cost

    @staticmethod
    def poisson_pmf(k, lam):
        """Poisson(k; lambda)"""
        from math import exp, factorial
        if k < 0:
            return 0.0
        return lam**k * exp(-lam) / factorial(k)


###############################################################################
# 2. Value Iteration
###############################################################################

def solve_value_function(mdp, tol=1e-8, max_iter=1000):
    """
    Value iteration: for each state, we choose the action that minimizes
    the expected one-period cost plus discounted future cost.

    Returns: V (value function array) and Q_fct (cost-to-go for each state-action).
    """
    nS = mdp.n_states
    nA = len(mdp.action_space)
    beta = mdp.beta

    # Initialize V(s) = 0
    V = np.zeros(nS)

    for iteration in range(max_iter):
        V_old = V.copy()

        # For each state s_idx, compute cost-to-go for each action
        Q_fct = np.zeros((nS, nA))
        for s_idx in range(nS):
            for a_idx, Q in enumerate(mdp.action_space):
                # immediate expected cost
                cost = mdp.cost_cache[(s_idx, Q)]
                # next-state distribution
                prob_next = mdp.trans_probs[(s_idx, Q)]
                # discounted expected future cost
                future_cost = beta * np.dot(prob_next, V_old)
                Q_fct[s_idx, a_idx] = cost + future_cost

        # Next iteration's V(s) = min over a of Q_fct(s,a)
        V = np.min(Q_fct, axis=1)

        # Check convergence
        if np.max(np.abs(V - V_old)) < tol:
            break

    # Return final V and Q_fct
    return V, Q_fct


###############################################################################
# 3. Choice Probabilities (Logit)
###############################################################################

def choice_probabilities(Q_fct, mu=1.0):
    """
    Suppose the decision-maker chooses actions with logit probabilities
    based on the *negative* of Q_fct (because Q_fct is cost-to-go,
    and we assume the agent 'dislikes cost').

    Probability that action a is chosen in state s ~
        exp( -Q_fct[s,a] / mu ) / sum_{a'} exp( -Q_fct[s,a'] / mu )
    """
    # We'll use "negative cost" as the "utility" in the logit exponent.
    negQ = -Q_fct / mu
    max_negQ = np.max(negQ, axis=1, keepdims=True)  # for numerical stability
    exp_negQ = np.exp(negQ - max_negQ)
    denom = np.sum(exp_negQ, axis=1, keepdims=True)
    P = exp_negQ / denom
    return P


###############################################################################
# 4. Log-Likelihood Function
###############################################################################

def log_likelihood(theta, data, i_min, i_max, Q_max, lambda_, beta=0.95, mu=1.0):
    """
    theta = [K, c, h, b] -> the cost parameters we want to estimate.

    data = list of (i_obs, Q_obs) over time.

    We'll:
      1) Construct MDP with these parameters
      2) Solve the DP for cost-to-go
      3) Compute logit choice probabilities
      4) Evaluate log-likelihood of observed (i_obs, Q_obs)
    """
    K, h, b = theta

    # Basic validity checks to keep params in a sensible range
    if K < 0 or h < 0 or b < 0:
        return 1e9  # big penalty

    # 1) Build the MDP
    mdp = InventoryBackorderMDP(
        i_min=i_min, i_max=i_max, Q_max=Q_max,
        lambda_=lambda_, beta=beta,
        K=K, c=1, h=h, b=b
    )

    # 2) Solve DP
    V, Q_fct = solve_value_function(mdp)

    # 3) Compute choice probabilities
    P_mat = choice_probabilities(Q_fct, mu=mu)  # shape (nS, nA)

    # 4) Evaluate log-likelihood
    ll = 0.0
    state_offset = mdp.i_min  # so that index = i_obs - i_min
    for (i_obs, Q_obs) in data:
        s_idx = i_obs - state_offset
        if s_idx < 0 or s_idx >= mdp.n_states:
            # if data is out of range, either skip or penalize
            ll += np.log(1e-12)
        else:
            Q_idx = Q_obs  # we matched action_space to [0..Q_max]
            if Q_idx < 0 or Q_idx > Q_max:
                ll += np.log(1e-12)
            else:
                # Probability that Q_obs was chosen in state i_obs
                p = P_mat[s_idx, Q_idx]
                ll += np.log(p + 1e-12)

    return -ll  # negative log-likelihood


###############################################################################
# 5. Data Simulation
###############################################################################

def simulate_data(mdp, T=1000, seed=42, mu=1.0):
    """
    Simulate time-series data (i_t, Q_t) from the MDP,
    assuming logit choice around the cost-to-go function.
    """
    np.random.seed(seed)

    # solve the DP
    V, Q_fct = solve_value_function(mdp)
    # get choice probabilities
    P_mat = choice_probabilities(Q_fct, mu=mu)

    data = []

    # Start from some initial inventory (e.g., 0)
    i_t = 0
    for t in range(T):
        # find index of i_t
        if i_t < mdp.i_min:
            i_t = mdp.i_min
        elif i_t > mdp.i_max:
            i_t = mdp.i_max

        s_idx = i_t - mdp.i_min
        # pick Q according to P_mat[s_idx, :]
        Q = np.random.choice(mdp.action_space, p=P_mat[s_idx])
        data.append((i_t, Q))

        # Realize demand
        D = sample_poisson(mdp.lambda_)

        # next inventory
        i_next = i_t + Q - D
        # allow it to go beyond i_min/i_max for simulation,
        # but re-clamp if it goes beyond extremes
        i_t = i_next

    return data

def sample_poisson(lam):
    """Simple Poisson sampler."""
    return np.random.poisson(lam)


###############################################################################
# 6. Parameter Estimation
###############################################################################

def estimate_parameters(
        data,
        i_min=-5,
        i_max=10,
        Q_max=5,
        lambda_=2.0,
        beta=0.95,
        mu=1.0
    ):
    """
    Estimate cost parameters [K, h, b] by MLE,
    given the observed (i, Q) data, using logit errors.
    """
    # objective for the optimizer
    def objective(theta):
        return log_likelihood(theta, data, i_min, i_max, Q_max, lambda_, beta=beta, mu=mu)

    # initial guess
    theta0 = np.array([3, 1, 2])  # [K, h, b]
    bnds = [(0,None), (0,None), (0,None)]  # all >= 0

    result = minimize(objective, theta0, method='L-BFGS-B', bounds=bnds)
    return result


###############################################################################
# 7. Demonstration (if run as script)
###############################################################################

if __name__ == "__main__":
    # "True" parameters for data generation
    true_K = 5.0
    true_c = 1
    true_h = 0.5
    true_b = 2.0
    beta = 0.95
    lambda_ = 4
    mu = 0.5   # logit scale (smaller => more deterministic)

    # Build MDP with the "true" parameters
    mdp_true = InventoryBackorderMDP(
        i_min=-5,
        i_max=20,
        Q_max=25,
        lambda_=lambda_,
        beta=beta,
        K=true_K,
        c=true_c,
        h=true_h,
        b=true_b
    )

    # Simulate data
    data = simulate_data(mdp_true, T=2000, seed=123, mu=mu)

    # Estimate parameters
    x, y = zip(*data)

    i_min = min(x)
    i_max = max(x)
    Q_max = max(y)


    est_result = estimate_parameters(
        data,
        i_min,
        i_max,
        Q_max,
        lambda_=lambda_,
        beta=beta,
        mu=mu
    )

    print("True        [K, h, b]:", true_K, true_h, true_b)
    print("Estimation Results:")
    print("  success:", est_result.success)
    print("  estimated [K, h, b]:", est_result.x)
    print("  negative log-likelihood:", est_result.fun)
    print("  message:", est_result.message)


True        [K, h, b]: 5.0 0.5 2.0
Estimation Results:
  success: True
  estimated [K, h, b]: [4.95986385 0.50891179 2.05483685]
  negative log-likelihood: 1940.6778383591218
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
